# Evaluation of Fine Tuned Whisper Small

In [ ]:
# Run this if required
%pip install transformers evaluate datasets librosa jiwer

  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Using cached evaluate-0.4.0-py3-none-any.whl (81 kB)
  Using cached datasets-2.12.0-py3-none-any.whl (474 kB)
  Using cached librosa-0.10.0.post2-py3-none-any.whl (253 kB)
  Using cached jiwer-3.0.1-py3-none-any.whl (21 kB)
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
  Using cached regex-2023.5.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (756 kB)
  Using cached tokenizers-0.13.3-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached multiprocess-0.70.14-py37-none-any.whl (115 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached xxhash-3.2.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (213 kB)
  Using cached pooch-1.6.0-py3-none-any.whl (56 kB)
  Using cached audioread-3.0.0-py3-none-any.whl
  Using cached soundfile-0.12.1-py2.py3-none-manylinux_2_31_x86_64.whl (1.2 MB)
  Usi

### Library Imports

In [ ]:
import csv
import torch
import evaluate

from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_from_disk

In [ ]:
torch.cuda.is_available()

True

### Model

In [ ]:
DEVICE = "cuda"

In [ ]:
# load model and processor
MODEL_PATH = "<path-to-model>"
processor = WhisperProcessor.from_pretrained(MODEL_PATH)
model = Whiprocessor = WhisperProcessor.from_pretrained(MODEL_PATH).to(DEVICE)
forced_decoder_ids = processor.get_decoder_prompt_ids(language="english", task="transcribe")


### Load Custom Dataset

In [ ]:
DATASET_PATH = "gs://cloud-ai-platform-e8edc327-855c-4911-bb8e-205517f8c899/asr/data/train/til_asr_base_train"

In [ ]:
ds = load_from_disk(DATASET_PATH)

In [ ]:
ds["train"][0]

{'path': 'audio/train_03701.wav',
 'annotation': 'THERE IS ONLY ONE WAY TO SUCCESS AND THAT IS HARD WORK AND DETERMINATION',
 'audio': {'path': 'train_03701.wav',
  'array': array([-3.96728516e-04, -5.18798828e-04, -4.88281250e-04, ...,
         -3.05175781e-05, -3.05175781e-05, -9.15527344e-05]),
  'sampling_rate': 16000}}

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['path', 'annotation', 'audio'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['path', 'annotation', 'audio'],
        num_rows: 750
    })
})

### Evaluation on 1 sample

In [ ]:
input_speech = ds["train"][0]["audio"]
input_features = processor(
    input_speech["array"],
    sampling_rate=input_speech["sampling_rate"],
    return_tensors="pt"
).input_features

# generate
generated_ids = model.generate(
    input_features.to(DEVICE),
    forced_decoder_ids=forced_decoder_ids,
)

# decode to text
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(transcription)

/opt/conda/lib/python3.7/site-packages/transformers/generation/utils.py:1350: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


 There is only one way to success and that is hard work and determination.


### Evaluation on entire dataset

In [ ]:
def normalize_to_til(transcript: str) -> str:
    # TIL output is purely uppercase alphabet and space
    # so we normalize the output to that
    result = "".join([c.upper() if c.isalpha() else " " for c in transcript])
    # Remove double spaces
    while "  " in result:
        result = result.replace("  ", " ")
    return result

In [ ]:
def map_to_pred(batch):
    audio = batch["audio"]
    raw = [i["array"] for i in audio]
    
    input_features = processor(raw, sampling_rate=audio[0]["sampling_rate"], return_tensors="pt").input_features
    batch["reference"] = [normalize_to_til(processor.tokenizer._normalize(transcript)) for transcript in batch["annotation"]]


    with torch.no_grad():
        predicted_ids = model.generate(input_features.to(DEVICE))
    preds = []
    for pred in predicted_ids:
      transcription = processor.decode(pred)
      preds.append(normalize_to_til(processor.tokenizer._normalize(transcription)))
    batch["prediction"] = preds
    return batch

preds = ds.map(map_to_pred, batched=True, batch_size=32)

# Calculate WER
wer = evaluate.load("wer")
test_result = wer.compute(predictions=preds["test"]["prediction"], references=preds["test"]["reference"])
train_result = wer.compute(predictions=preds["train"]["prediction"], references=preds["train"]["reference"])
print(f"Train WER: {train_result * 100:.3f}")
print(f"Test WER: {test_result * 100:.3f}")

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

Train WER: 3.665
Test WER: 3.489


In [ ]:
def map_to_pred_eval(batch):
    audio = batch["audio"]
    raw = [i["array"] for i in audio]
    
    input_features = processor(raw, sampling_rate=audio[0]["sampling_rate"], return_tensors="pt").input_features
    with torch.no_grad():
        predicted_ids = model.generate(input_features.to(DEVICE))
    preds = []
    for pred in predicted_ids:
      transcription = processor.decode(pred)
      preds.append(normalize_to_til(processor.tokenizer._normalize(transcription)))
    batch["prediction"] = preds
    return batch

In [ ]:
EVAL_DATASET = "gs://cloud-ai-platform-e8edc327-855c-4911-bb8e-205517f8c899/asr/data/test/til_asr_base_eval"
eval_ds = load_from_disk(EVAL_DATASET)
eval_ds

Dataset({
    features: ['path', 'audio'],
    num_rows: 12000
})

In [ ]:
eval_preds = eval_ds.map(map_to_pred_eval, batched=True, batch_size=32)

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

In [ ]:
def create_csv(paths, predictions, output_filename):
    # Ensure that paths and predictions are of the same length
    assert len(paths) == len(predictions), "Lists must have the same length"

    with open(output_filename, 'w', newline='') as csvfile:
        fieldnames = ['path', 'annotation']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for path, prediction in zip(paths, predictions):
            writer.writerow({'path': path[6:], 'annotation': prediction})

# Test with example data
paths = eval_ds["path"]
predictions = eval_ds["prediction"]

create_csv(paths, predictions, 'eval_submission_whisper_small_zero_shot_defaultdecoding.csv')

## Next Steps
- Error Analysis
- Denoising of Data